# Automatic Essay Scoring

In [2]:
import numpy as np
import pandas as pd
import gensim 
from gensim.models.doc2vec import Doc2Vec
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score

stopwords = set(stopwords.words('english'))

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


First, we can set up the dataframes and explore the data. We will drop columns that we don't need and those with NaN values. There was one row without a domain1_score, which I removed. Some essays also contained domain2 or domain3 scores, but since not all the data has that field, I will ignore that for now.

In [3]:
data = pd.ExcelFile('./data/training_set_rel3.xls')
df = data.parse("training_set")
df = df.drop('rater1_domain1', 1)
df = df.drop('rater2_domain1', 1)
df = df.dropna(axis = 1)

df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [16]:
essays = df['essay']
essays[0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

Here, we can see that personally identifiying information has been replaces with @NER where NER is a NER tag. We can remove these symbols to avoid interferring with the spell checking counts.

In [9]:
# Function to get all text from each essay - to build doc2vec
def all_essays(df):
    for (i, essay) in enumerate(df['essay']):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(essay), [i])
        

all_essay_lst = all_essays(df)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(all_essay_lst)
%time model.train(all_essay_lst, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 51.5 ms, sys: 25.2 ms, total: 76.7 ms
Wall time: 76.1 ms


Features #TODO write description of features

In [7]:
# Setup pre-trained word2vec model
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)

In [10]:
num_rows = df.shape[0]
essays = df['essay'].values

#initialize dataframe columns
df['word_count'] = np.nan 
df['sentence_count'] = np.nan
df['avg_word_length'] = np.nan 
df['num_exclamation_marks'] = np.nan
df['num_question_marks'] = np.nan
df['num_stopwords'] = np.nan
df['word2vec_concat'] = np.nan

df['verb_count'] = np.nan
df['foreign_count'] = np.nan
df['adj_count'] = np.nan
df['conj_count'] = np.nan


# TODO
#df['num_advanced_words'] = np.nan
#df['spelling_errors'] = np.nan

def replace_punc(text):
    return text.replace("@", "").replace("%", "")

def get_pos_tags(essay):
    nouns = verbs = foreign = adj = adv = conj = 0
    tokens = nltk.word_tokenize(essay)
    for token in tokens:
        pos_tag = nltk.pos_tag(nltk.word_tokenize(token))
        for (_, tag) in (pos_tag):
            if tag[0] == "N":
                nouns += 1
            elif tag[0] == "V":
                verbs += 1
            elif tag[0:2] == "FW":
                foreign += 1
            elif tag[0] == "J":
                adj += 1
            elif tag[0] == "R":
                adv += 1
            elif tag[0:2] == "CC" or tag[0:2] == "IN":
                conj += 1
    
    return [nouns, verbs, foreign, adj, adv, conj]


for i in range(num_rows):
    
    # Remove placeholders
    text = replace_punc(essays[i])
    
    # Turn essay into list of words
    text = essays[i].split(" ")
    
    # Set word count
    df.set_value(i,'word_count', len(text))
    
    # Sentence count
    df.set_value(i, 'sentence_count', len(nltk.tokenize.sent_tokenize(essays[i])))
    
    # Average word length
    word_len = sum(len(word) for word in text) / len(text)
    df.set_value(i, 'avg_word_length', word_len)
    
    # Number of exclamation marks
    df.set_value(i, "num_exclamation_marks", sum(word.count("!") for word in essays[i]))
    
    # Number of question marks
    df.set_value(i, "num_question_marks", sum(word.count("?") for word in essays[i]))
    
    # Number of stop words
    df.set_value(i, "num_stopwords", sum([1 for word in text if word.lower() in stopwords]))

    # Word2Vec conversion - min + max
    df.set_value(i, 'word2vec_concat', min(model.docvecs[i]) + max(model.docvecs[i]))
    
    # POS tag counts
    pos_lst = get_pos_tags(essays[i])
    df.set_value(i,'verb_count', pos_lst[1])
    df.set_value(i,'foreign_count', pos_lst[2])
    df.set_value(i,'adj_count', pos_lst[3])
    df.set_value(i,'conj_count', pos_lst[5])

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:57: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:60: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:64: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:67: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:70: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel/_

In [11]:
def get_tfidf_vectors(essays):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.9, min_df=5, max_features=50, stop_words="english", binary=True)
    tfidf_vectors = vectorizer.fit_transform(essays)
    new_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names())
    
    return pd.concat([df, new_df], axis=1)

df = get_tfidf_vectors(essays)
df.head()

,essay_id,essay_set,essay,domain1_score,word_count,sentence_count,avg_word_length,num_exclamation_marks,num_question_marks,num_stopwords,...,state,story,thing,things,think,time,use,want,way,world
0,1,1,"Dear local newspaper, I think effects computer...",8,338.0,16.0,4.550296,4.0,2.0,168.0,...,0.0,0.0,0.234392,0.202706,0.193065,0.180893,0.247159,0.225527,0.196461,0.000000
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,419.0,20.0,4.463007,1.0,1.0,189.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.259524,0.000000,0.000000,0.269336
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,279.0,14.0,4.526882,0.0,0.0,140.0,...,0.0,0.0,0.000000,0.000000,0.217546,0.203830,0.278498,0.254123,0.221372,0.289028
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,524.0,27.0,5.041985,2.0,1.0,222.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.258546,0.000000,0.000000,0.268321
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,465.0,30.0,4.526882,0.0,0.0,236.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.240730,0.209705,0.273795


### Text data

We need to convert the essay strings into some numerical form. We could use Word2Vec, a TF-IDF Vectorizer, etc.

We have added all the features, so now we can start to explore correlations between features and scores (to ensure we are making correct assumptions and to discover potential new features), and perform the logistic regression.

In [17]:
'''         
Worsened: 'word2vec_avg', 'noun_count','adj_count', 'adv_count', 'foreign_count',
'''

#features = df.drop('domain1_score', axis=1)
x = df.drop(['domain1_score', 'essay'], axis=1)
y = df['domain1_score']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
logistic_reg = LogisticRegression()
logistic_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
predictions = logistic_reg.predict(X_test)
print('Logistic regression classifier accuracy: {:.2f}'.format(logistic_reg.score(X_test, y_test)))

Logistic regression classifier accuracy: 0.46


In [19]:
print(cohen_kappa_score(predictions, y_test, weights="quadratic"))

0.7985546474708368
